![cactus](http://cactuscode.org/global/images/cactuslogo.png)
# Compiling Cactus!
Step 1 is to download the code. We make a directory for the tutorial and compile the thornlist provided by this tutorial.

In [ ]:
%cd ~/
%mkdir Tutorial
%cd ~/Tutorial
!curl -kLO https://raw.githubusercontent.com/gridaphobe/CRL/ET_2018_02/GetComponents
!chmod a+x ./GetComponents
!echo no|./GetComponents --parallel ~/PreSyncTutorial/thorns.th

This is the command to build Cactus using our thornlist. As written, it will build in parallel using two processes.

In [ ]:
%cd ~/Tutorial/CactusPre
!./simfactory/bin/sim setup-silent
!./simfactory/bin/sim build -j 2 --thornlist=~/Tutorial/CactusPre/thornlists/thorns.th

<h1>Running Cactus!</h1>

The thornlist we used pulls the presync branch of all the thorns, and PreSync is fully active. However, backward compatibility is critical for any framework update. Therefore, we will show that old code still functions with these branches of the Toolkit.

Below, we run a simple 2D wave with symmetric boundary conditions. This parameter file uses the thorn OldWave, which is a thorn designed for the Einstein Toolkit without PreSync. This is in the PresyncWave repository, which also contains the thorn PresyncWave. PresyncWave uses PreSync and is also capable of running with backward compatible settings that ignores PreSync. For this tutorial, we will be changing OldWave to match PresyncWave.

In [ ]:
%mkdir ~/Tutorial/CactusPre/pars

In [ ]:
%%writefile ~/Tutorial/CactusPre/pars/oldwave.par
ActiveThorns = "OldWave Boundary"
ActiveThorns = "CoordBase Carpet CartGrid3D MoL SymBase"
ActiveThorns = "CarpetReduce CarpetIOBasic CarpetIOASCII Time"

cactus::cctk_itlast = 50

CartGrid3D::type = "coordbase"
CartGrid3D::avoid_origin = "no"
CoordBase::domainsize = "minmax"
CoordBase::spacing    = "gridspacing"
CoordBase::xmin = -5
CoordBase::xmax = 5
CoordBase::ymin = -5
CoordBase::ymax = 5
CoordBase::zmin = 0
CoordBase::zmax = 0
CoordBase::dx = 0.2
CoordBase::dy = 0.2
CoordBase::boundary_size_z_lower = 0
CoordBase::boundary_size_z_upper = 0
CoordBase::boundary_shiftout_z_lower = 1
CoordBase::boundary_shiftout_z_upper = 1

Time::dtfac = 0.2

Carpet::domain_from_coordbase = "yes"
Carpet::ghost_size_x = 1
Carpet::ghost_size_y = 1
Carpet::ghost_size_z = 0

IO::out_dir = $parfile

IOBasic::outInfo_every = 5
IOBasic::outInfo_vars =  "OldWave::psi"

IOASCII::out2D_every = 5
IOASCII::out2D_xyplane_z = 0
IOASCII::out2D_vars = "
  OldWave::energy
  OldWave::psi
  OldWave::phi
"

CarpetIOASCII::compact_format = true
IOASCII::output_ghost_points = "no"

# MoL
MoL::ODE_Method = "RK3"

OldWave::BCtype = "symmetry"

This next cell deletes our simulation in case we want to throw it away and start over again for some reason.

In [ ]:
!rm -fr ~/simulations/oldwave

Enough already! Let's run Cactus!

In [ ]:
%cd ~/Tutorial/CactusPre
!./simfactory/bin/sim create-run --procs 2 --num-threads 1 pars/oldwave.par

Data can be found in this directory. Using the next couple of commands, we will browse it.

In [ ]:
%cd ~/simulations/oldwave/output-0000/oldwave

In [ ]:
%ls *.asc

In [ ]:
# This cell enables inline plotting in the notebook
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
# https://matplotlib.org/examples/color/colormaps_reference.html
cmap = cm.gist_rainbow

In [ ]:
file_data = np.genfromtxt("psi.xy.asc")
sets = np.unique(file_data[:,0])
width = 8
height = 4
print("sets=",sets)
mn, mx = np.min(file_data[:,8]),np.max(file_data[:,8])
for which in sets: 
    print("which=",which)
    g = file_data[file_data[:,0]==which,:]
    x = g[:,5]
    y = g[:,6]
    z = g[:,8]
    zi = z.reshape(len(np.unique(y)),len(np.unique(x)))
    print('min/max=',np.min(zi),np.max(zi))
    plt.figure(figsize=(width, height))
    plt.imshow(zi[::-1,:],cmap,clim=(mn,mx))
    plt.show()

<h2>Using PreSync</h2>
We have confirmed that OldWave works, and by extension that legacy code still functions with the PreSync update to Cactus. Our next task is to change OldWave to use PreSync and remain backwards compatible with code that has not been updated to use PreSync.

To begin, we will change the schedule.ccl to use PreSync while remaining backward compatible. First, let us examine the current schedule.

In [ ]:
%pycat ~/Tutorial/CactusPre/repos/PresyncWave/OldWave/schedule.ccl

Currently, scheduled functions have SYNC statements to trigger synchronization, and the Boundary Routines section handles the registration and application of boundary conditions. PreSync instead relies on read/write declarations to determine when synchronization and boundary condition application routines should run. PreSync ignores the SYNC statements, so we can safely leave them for backward compatibility and add the READS and WRITES statements.

Read/write statements require the thorn::variable that is accessed and also the region on which it is accessed. PreSync supports several regions of validity, but for most functions the only two which should be used are "interior" and "everywhere". A variable which is read "everywhere" requires that the variable be valid in the ghost zones and boundary regions, while reading the interior does not. Writing variables changes their region of validity, causing synchronization to trigger when the next read(everywhere) is encountered.

As an example, let us look at the function presync_wave_init. It is scheduled as

<div style='border-style: solid; border-color: black; padding: 5px;'>
<pre>
schedule oldsync_wave_init at CCTK_POSTINITIAL
{
  LANG: C
  SYNC: OldWave::evo_vars
} "initial condition"
</pre>
</div>

This simple function writes psi and phi initial data, and it reads the grid variables x, and y. The code only loops over the interior, so Therefore, we may change the scheduling to the following:

<div style='border-style: solid; border-color: black; padding: 5px;'>
<pre>
schedule oldsync_wave_init at CCTK_POSTINITIAL
{
  LANG: C
  READS: Grid::x(interior), y
  WRITES: OldWave::evo_vars(interior)
  SYNC: OldWave::evo_vars
} "initial condition"
</pre>
</div>

As seen in the READ declaration, multiple variables can be listed on one line by adding them after the region specification. Also, a group may be given instead of individual variables, as in the WRITE declaration.

Compared to the confusing and non-trivial problem of determining the correct positions of the SYNC statements, determining the read/write declarations for a function is far simpler and only requires knowledge of the one scheduled function (and any functions it calls).

All the functions in OldWave have very simple read/write dependencies, and adding those to the schedule.ccl are all that is required to properly trigger synchronization with PreSync. To see the complete declarations, you may look further down where we write the new schedule.ccl.

For now, we move on to boundary conditions. This thorn uses its own boundary conditions, which it registers with the Boundary thorn in the presync_registervars function. To apply boundary conditions, it schedules a SelectBCs function and the ApplyBCs group (provided by Boundary) wherever boundary conditions should be applied. In this case, there are two separate SelectBCs functions, and they must be scheduled separately.

In contrast, PreSync only requires the SelectBCs routine to run once, at the beginning of a simulation. This is facillitated by the Boundary2 thorn, which provides the features of Boundary updated to use PreSync. In addition, it provides groups for boundary registration and selection. The new schedule for the boundary routines is
<div style='border-style: solid; border-color: black; padding: 5px;'>
<pre>
schedule presync_registerboundary in PreSync_Registration
{
  LANG: C
} "register boundaries"

schedule presync_SelectBCs in PreSync_Selection
{
  LANG: C
} "select boundary conditions"

schedule energy_SelectBCs in PreSync_Selection
{
  LANG: C
} "select boundary conditions"
</pre>
</div>

While the selectBCs functions could be combined, they remain separate to provide backward compatibility. However, we can't simply add this to the schedule.ccl and expect it to work. We need to turn off the new scheduled routines and turn on the old scheduled routines if running in backward-compatible mode. This is accomplished by using the routine
<div style='border-style: solid; border-color: black; padding: 5px;'>
<pre>
CCTK_ParameterValInt("use_psync","Carpet")
</pre>
</div>
to determine which should run. If it equals 1, then PreSync is active. If it equals 0, then the backward-compatible routines should be used.

The changes discussed above are implemented in the schedule.ccl below. This command will overwrite the old schedule.ccl with our new version.

In [ ]:
%%writefile ~/Tutorial/CactusPre/repos/PresyncWave/OldWave/schedule.ccl
# Schedule definitions for thorn OldWave
storage: rhs_vars[3], evo_vars[3], evo_divs, wave_energy[3]

schedule oldsync_wave_init at CCTK_POSTINITIAL
{
  LANG: C
  READS: Grid::x(interior), y
  WRITES: OldWave::evo_vars(interior)
  SYNC: OldWave::evo_vars
} "initial condition"

schedule oldsync_wave_evolve in MoL_CalcRHS
{
  LANG: C
  READS: OldWave::evo_divs(interior), phi
  WRITES: OldWave::rhs_vars(interior)
  SYNC: OldWave::rhs_vars
} "Evolve loop"

schedule oldsync_derivatives in MoL_CalcRHS before oldsync_wave_evolve
{
  LANG: C
  READS: OldWave::evo_vars(everywhere)
  WRITES: OldWave::evo_divs(interior)
} "Compute derivatives"

schedule oldsync_registervars in MoL_Register
{
  LANG: C
  OPTIONS: META
}"Register funwave variables for MoL"

#################################################
##             Boundary Routines               ##
#################################################

# In this function, we register boundary conditions
# with Carpet (not thorn Boundary). This new way
# of doing things makes it possible for Carpet to
# fill in the exterior of the variable at the
# same time it performs a sync.
if(CCTK_ParameterValInt("use_psync","Carpet") == 1) {
  schedule oldsync_registerboundary in PreSync_Registration
  {
    LANG: C
  } "register boundaries"

  schedule oldsync_evolve_SelectBCs in PreSync_Selection
  {
    LANG: C
  } "select boundary conditions"

  schedule oldsync_energy_SelectBCs in PreSync_Selection
  {
    LANG: C
  } "select boundary conditions"
}

if(CCTK_ParameterValInt("use_psync","Carpet") == 0) {
  schedule group PSWave_Boundaries in MoL_CalcRHS before oldsync_derivatives
  {
  } "boundary condition group"
  schedule group PSWave_Boundaries in CCTK_ANALYSIS before oldsync_energy
  {
  } "boundary condition group"

  schedule group PSWave_Boundaries at POSTRESTRICT
  {
  } "boundary condition group"

  schedule oldsync_registerboundary at CCTK_WRAGH
  {
    LANG: C
  } "register boundaries"

  schedule oldsync_evolve_SelectBCs in PSWave_Boundaries
  {
    LANG: C
  } "select boundary conditions"

  schedule GROUP ApplyBCs as PSWave_ApplyBCs in PSWave_Boundaries after oldsync_evolve_selectBCs
  {
  } "Apply boundary conditions"

  schedule group Energy_Boundary at CCTK_ANALYSIS after oldsync_energy
  {
  } "boundary condition group"

  schedule oldsync_energy_SelectBCs in Energy_Boundary
  {
    LANG: C
  } "select boundary conditions"

  schedule GROUP ApplyBCs as Energy_ApplyBCs in Energy_Boundary after oldsync_energy_SelectBCs
  {
  } "Apply boundary conditions"
}

#################################################
##            Energy Calculations              ##
#################################################

schedule oldsync_energy at CCTK_ANALYSIS
{
  LANG: C
  READS: OldWave::evo_vars(everywhere)
  WRITES: OldWave::energy(interior)
  SYNC: OldWave::wave_energy
} "Calculate energy"

We now have a working schedule.ccl. None of the other ccl files need to be changed, so we can move on to the source code. First, we need to look at the PSWave.h. This provides the function type used to pass the boundary routines to the Boundary thorn. To do this, we could have used the Boundary.h thorn from Boundary or included the typedef itself. We elected to perform the latter, as we can see below.

In [ ]:
%cat ~/Tutorial/CactusPre/repos/PresyncWave/OldWave/src/PSWave.h

For PreSync, we replace this with the header file PreSync.h. This header file is located in the flesh, so it is readily available to all thorns. It contains the boundary_function typedef as well as the definitions for the regions of validity. In addition to variable registration, this is included whenever a function needs to access the validity of a variable.

In [ ]:
%%writefile ~/Tutorial/CactusPre/repos/PresyncWave/OldWave/src/PSWave.h
#ifndef _PSWave_H_
#define _PSWave_H_

#include "PreSync.h"

#ifdef __cplusplus
extern "C" {
#endif

/* prototype for routine registered as providing 'zero' boundary condition */
CCTK_INT fun_stwave(const cGH *cctkGH, CCTK_INT num_vars, CCTK_INT *var_indices,
                  CCTK_INT *faces, CCTK_INT *widths, CCTK_INT *table_handles);

/* prototype for routine registered as providing 'symmetry' and 'anti-symmetry' boundary conditions */
CCTK_INT fun_bf2(const cGH *cctkGH, CCTK_INT num_vars, CCTK_INT *var_indices,
                  CCTK_INT *faces, CCTK_INT *widths, CCTK_INT *table_handles);

#ifdef __cplusplus
}
#endif

#endif /* _PSWave_H_ */

The next step is very easy, though it will be disproportionately long in the notebook. The current method for accessing variables in Cactus is to use the DECLARE_CCTK_ARGUMENTS macro. However, this declares *every* variable and not just those needed by the function. To reduce these declarations and provide additional error-checking, PreSync provides new (optional) macros which are generated based on the read/write declarations. Each scheduled function has its own macro which only provides access to declared variables. In addition, read-only declarations are declared as const, which provides additional verification for read/write declarations. To include these new macros, we add
```
#include "cctk_Arguments_Checked.h"
```
and replace DECLARE_CCTK_ARGUMENTS with DECLARE_CCTK_ARGUMENTS_FunctionName. This is done below for all the code in the OldWave thorn.

In [ ]:
%%writefile ~/Tutorial/CactusPre/repos/PresyncWave/OldWave/src/energy.cc
#include "cctk.h" 
#include "cctk_Arguments.h"
#include "cctk_Arguments_Checked.h"
#include "cctk_Parameters.h"
#include "iostream"

void oldsync_energy(CCTK_ARGUMENTS)
{
  DECLARE_CCTK_ARGUMENTS_oldsync_energy
  DECLARE_CCTK_PARAMETERS;
      
  const int imin0=cctk_nghostzones[0];
  const int imin1=cctk_nghostzones[1];
  const int imin2=cctk_nghostzones[2];
  const int imax0=cctk_lsh[0] - cctk_nghostzones[0];
  const int imax1=cctk_lsh[1] - cctk_nghostzones[1];
  const int imax2=cctk_lsh[2] - cctk_nghostzones[2];
  const int zero = CCTK_GFINDEX3D(cctkGH,0,0,0);
  const int di = (cctk_lsh[0]==1) ? 0:CCTK_GFINDEX3D(cctkGH,1,0,0) - zero;
  const int dj = (cctk_lsh[1]==1) ? 0:CCTK_GFINDEX3D(cctkGH,0,1,0) - zero;
  const int dk = (cctk_lsh[2]==1) ? 0:CCTK_GFINDEX3D(cctkGH,0,0,1) - zero;
  #pragma omp parallel
  CCTK_LOOP3(calc_energy,i,j,k,
    imin0,imin1,imin2,imax0,imax1,imax2,
    cctk_ash[0],cctk_ash[1],cctk_ash[2])
  {
    int cc = CCTK_GFINDEX3D(cctkGH,i,j,k);
    double psix = (psi[cc+di]-psi[cc-di])/(2.0*CCTK_DELTA_SPACE(0));
    double psiy = (psi[cc+dj]-psi[cc-dj])/(2.0*CCTK_DELTA_SPACE(1));
    double psiz = (psi[cc+dk]-psi[cc-dk])/(2.0*CCTK_DELTA_SPACE(2));
    energy[cc] = phi[cc]*phi[cc] + psix*psix +
      psiy*psiy + psiz*psiz;
  }
  CCTK_ENDLOOP3(calc_energy);
}

In [ ]:
%%writefile ~/Tutorial/CactusPre/repos/PresyncWave/OldWave/src/Wave.cc
#include <cctk.h>
#include <cctk_Arguments.h>
#include <cctk_Arguments_Checked.h>
#include <cctk_Parameters.h>
#include <iostream>

#define sq(X) (X)*(X)

extern "C"
void oldsync_wave_init(CCTK_ARGUMENTS)
{
  DECLARE_CCTK_ARGUMENTS_oldsync_wave_init
  DECLARE_CCTK_PARAMETERS

  const int imin0=cctk_nghostzones[0];
  const int imin1=cctk_nghostzones[1];
  const int imin2=cctk_nghostzones[2];
  const int imax0=cctk_lsh[0] - cctk_nghostzones[0];
  const int imax1=cctk_lsh[1] - cctk_nghostzones[1];
  const int imax2=cctk_lsh[2] - cctk_nghostzones[2];
  CCTK_REAL x0 = x[CCTK_GFINDEX3D(cctkGH,cctk_lsh[0]/2,cctk_lsh[1]/2,cctk_lsh[2]/2)];
  CCTK_REAL y0 = x[CCTK_GFINDEX3D(cctkGH,cctk_lsh[0]/2,cctk_lsh[1]/2,cctk_lsh[2]/2)];
  #pragma omp parallel
  CCTK_LOOP3(calc_oldsync_wave_init,
    i,j,k, imin0,imin1,imin2, imax0,imax1,imax2,
    cctk_ash[0],cctk_ash[1],cctk_ash[2])
  {
    int cc = CCTK_GFINDEX3D(cctkGH,i,j,k);
    psi[cc] = exp(-sq(x[cc]-x0)-sq(y[cc]-y0));
    phi[cc] = 0;
  }
  CCTK_ENDLOOP3(calc_oldsync_wave_init);
}

extern "C"
void oldsync_wave_evolve(CCTK_ARGUMENTS)
{
  DECLARE_CCTK_ARGUMENTS_oldsync_wave_evolve;
  DECLARE_CCTK_PARAMETERS;

  const int imin0=cctk_nghostzones[0];
  const int imin1=cctk_nghostzones[1];
  const int imin2=cctk_nghostzones[2];
  const int imax0=cctk_lsh[0] - cctk_nghostzones[0];
  const int imax1=cctk_lsh[1] - cctk_nghostzones[1];
  const int imax2=cctk_lsh[2] - cctk_nghostzones[2];
  #pragma omp parallel
  CCTK_LOOP3(calc_oldsync_wave_evol,
    i,j,k, imin0,imin1,imin2, imax0,imax1,imax2,
    cctk_ash[0],cctk_ash[1],cctk_ash[2])
  {
    int cc = CCTK_GFINDEX3D(cctkGH,i,j,k);
    psi_rhs[cc] = phi[cc];
    phi_rhs[cc] = dxx_psi[cc]+dyy_psi[cc]+dzz_psi[cc];
  }
  CCTK_ENDLOOP3(calc_oldsync_wave_evol);
}

extern "C"
void oldsync_derivatives(CCTK_ARGUMENTS)
{
  DECLARE_CCTK_ARGUMENTS_oldsync_derivatives;
  DECLARE_CCTK_PARAMETERS;

  const int imin0=cctk_nghostzones[0];
  const int imin1=cctk_nghostzones[1];
  const int imin2=cctk_nghostzones[2];
  const int imax0=cctk_lsh[0] - cctk_nghostzones[0];
  const int imax1=cctk_lsh[1] - cctk_nghostzones[1];
  const int imax2=cctk_lsh[2] - cctk_nghostzones[2];
  const int zero = CCTK_GFINDEX3D(cctkGH,0,0,0);
  const int di = (cctk_lsh[0]==1) ? 0:CCTK_GFINDEX3D(cctkGH,1,0,0) - zero;
  const int dj = (cctk_lsh[1]==1) ? 0:CCTK_GFINDEX3D(cctkGH,0,1,0) - zero;
  const int dk = (cctk_lsh[2]==1) ? 0:CCTK_GFINDEX3D(cctkGH,0,0,1) - zero;
  assert(!std::isnan(psi[zero]));
  #pragma omp parallel
  CCTK_LOOP3(calc_oldsync_derivs,
    i,j,k, imin0,imin1,imin2, imax0,imax1,imax2,
    cctk_ash[0],cctk_ash[1],cctk_ash[2])
  {
    int cc = CCTK_GFINDEX3D(cctkGH,i,j,k);
    dxx_psi[cc] = (psi[cc+di]+psi[cc-di]-2.0*psi[cc])/(CCTK_DELTA_SPACE(0)*CCTK_DELTA_SPACE(0));
    dyy_psi[cc] = (psi[cc+dj]+psi[cc-dj]-2.0*psi[cc])/(CCTK_DELTA_SPACE(1)*CCTK_DELTA_SPACE(1));
    dzz_psi[cc] = (psi[cc+dk]+psi[cc-dk]-2.0*psi[cc])/(CCTK_DELTA_SPACE(2)*CCTK_DELTA_SPACE(2));
  }
  CCTK_ENDLOOP3(calc_oldsync_derivs);
}

Congratulations! You've just changed your first thorn to use PreSync! To verify that we haven't broken backward compatibility, we can recompile and run the test in OldWave. The parameter file for this test is identical to the one we used earlier, except it calculates less iterations.

In [ ]:
%cd ~/Tutorial/CactusPre/
!./simfactory/bin/sim build -j 2 --thornlist=~/Tutorial/CactusPre/thornlists/thorns.th

In [ ]:
%cd ~/Tutorial/CactusPre/
!./simfactory/bin/sim create-run mytest --testsuite --procs 2 --num-threads 1 --select-tests PresyncWave/OldWave

We see that the oldsync test in OldWave still passes with our changes to the code. The default settings in PreSync use the old synchronization method to preserve backward compatibility with old parameter files. Therefore, this is using the old synchronization method. We can actually use some of the error-checking added in PreSync without activating PreSync. To do this, we just add a single parameter to the parameter file,
```
Carpet::psync_error = "yes"
```
which turns on the features which produce errors if a function tries to access variables which are not valid. This requires that all the code being used has the READ/WRITE declarations, but it still uses the old SYNC statements for triggering synchronization and the old Boundary thorn's boundary condition application.

In [ ]:
%%writefile ~/Tutorial/CactusPre/repos/PresyncWave/OldWave/test/oldsync.par
ActiveThorns = "OldWave Boundary"
ActiveThorns = "CoordBase Carpet CartGrid3D MoL SymBase"
ActiveThorns = "CarpetReduce CarpetIOBasic CarpetIOASCII Time"

cactus::cctk_itlast = 25

CartGrid3D::type = "coordbase"
CartGrid3D::avoid_origin = "no"
CoordBase::domainsize = "minmax"
CoordBase::spacing    = "gridspacing"
CoordBase::xmin = -5
CoordBase::xmax = 5
CoordBase::ymin = -5
CoordBase::ymax = 5
CoordBase::zmin = 0
CoordBase::zmax = 0
CoordBase::dx = 0.2
CoordBase::dy = 0.2
CoordBase::boundary_size_z_lower = 0
CoordBase::boundary_size_z_upper = 0
CoordBase::boundary_shiftout_z_lower = 1
CoordBase::boundary_shiftout_z_upper = 1

Time::dtfac = 0.2

Carpet::domain_from_coordbase = "yes"
Carpet::ghost_size_x = 1
Carpet::ghost_size_y = 1
Carpet::ghost_size_z = 0

Carpet::psync_error = "yes"

IO::out_dir = $parfile

IOBasic::outInfo_every = 5
IOBasic::outInfo_vars =  "OldWave::psi"

IOASCII::out2D_every = 5
IOASCII::out2D_xyplane_z = 0
IOASCII::out2D_vars = "
  OldWave::energy
  OldWave::psi
  OldWave::phi
"

CarpetIOASCII::compact_format = true
IOASCII::output_ghost_points = "no"

# MoL
MoL::ODE_Method = "RK3"

OldWave::BCtype = "symmetry"

Before we re-run the test to check that this parameter file works, we should add a new test that uses PreSync as well. First, we can copy the old test's output data and parameter file. Then, we can turn on the PreSync parameters to activate it. Finally, we change the active thorn Boundary to Boundary2. This thorn provides the same boundary conditions as Boundary, but it uses PreSync.

In [ ]:
%cd ~/Tutorial/CactusPre/repos/PresyncWave/OldWave/test
%cp -r ./oldsync/ ./presync/ 
%cp ./oldsync.par ./presync.par

In [ ]:
%%writefile ~/Tutorial/CactusPre/repos/PresyncWave/OldWave/test/presync.par
ActiveThorns = "OldWave Boundary2"
ActiveThorns = "CoordBase Carpet CartGrid3D MoL SymBase"
ActiveThorns = "CarpetReduce CarpetIOBasic CarpetIOASCII Time"

cactus::cctk_itlast = 25

CartGrid3D::type = "coordbase"
CartGrid3D::avoid_origin = "no"
CoordBase::domainsize = "minmax"
CoordBase::spacing    = "gridspacing"
CoordBase::xmin = -5
CoordBase::xmax = 5
CoordBase::ymin = -5
CoordBase::ymax = 5
CoordBase::zmin = 0
CoordBase::zmax = 0
CoordBase::dx = 0.2
CoordBase::dy = 0.2
CoordBase::boundary_size_z_lower = 0
CoordBase::boundary_size_z_upper = 0
CoordBase::boundary_shiftout_z_lower = 1
CoordBase::boundary_shiftout_z_upper = 1

Time::dtfac = 0.2

Carpet::domain_from_coordbase = "yes"
Carpet::ghost_size_x = 1
Carpet::ghost_size_y = 1
Carpet::ghost_size_z = 0

Carpet::use_psync = "yes"
Carpet::psync_only = "yes"
Carpet::psync_error = "yes"

IO::out_dir = $parfile

IOBasic::outInfo_every = 5
IOBasic::outInfo_vars =  "OldWave::psi"

IOASCII::out2D_every = 5
IOASCII::out2D_xyplane_z = 0
IOASCII::out2D_vars = "
  OldWave::energy
  OldWave::psi
  OldWave::phi
"

CarpetIOASCII::compact_format = true
IOASCII::output_ghost_points = "no"

# MoL
MoL::ODE_Method = "RK3"

OldWave::BCtype = "symmetry"

Now, we add this new test to the test.ccl.

In [ ]:
%%writefile ~/Tutorial/CactusPre/repos/PresyncWave/OldWave/test/test.ccl
TEST oldsync
{
}

TEST presync
{
}

To review, we added PreSync error-checking to the oldsync test, and we added the presync test. The presync test uses the output from oldsync because it should output the same results if it is working correctly.

In [ ]:
%cd ~/Tutorial/CactusPre/
!./simfactory/bin/sim create-run testsuite --testsuite --procs 2 --num-threads 1 --select-tests PresyncWave/OldWave

You should see that both tests pass. As a final note, there is one more useful feature added in PreSync. We can now see how many synchronization calls occur in a given run. We can compare the number of synchronization calls in each by extracting the synchronization count from the log files.

In [ ]:
%cd ~/simulations/testsuite/output-0000/TEST/sim/OldWave

import re

log1 = open("oldsync.log", "r")
log2 = open("presync.log", "r")

for line in log1:
    if re.match("(.*)variable synchronizations occurred(.*)", line):
        m = re.match("(.*) ([0-9]+) (.*)", line)
        print("oldsync synchronized",m.group(2),"times")
for line in log2:
    if re.match("(.*)variable synchronizations occurred(.*)", line):
        m = re.match("(.*) ([0-9]+) (.*)", line)
        print("presync synchronized",m.group(2),"times")


Interestingly, the PreSync-enabled run has less synchronizations. This has been found to be a consistent feature of the new update, and most tests in the testsuite take ~20% less time to run due to this change. Since PreSync is not yet compatible with Carpet AMR, the exact speed-up of PreSync for full-scale simulations is not determinable at this time. Still, these results show that PreSync provides significant improvements to the Cactus Framework.

<table><tr><td>This work sponsored by NSF grants <a href="https://www.nsf.gov/awardsearch/showAward?AWD_ID=1550551"> OAC 1550551</a>  <a href="https://www.nsf.gov/awardsearch/showAward?AWD_ID=1539567"><td><img src="https://www.nsf.gov/awardsearch/images/common/nsf_logo_bottom.png"></tr></table>